In [ ]:
!pip -q uninstall -y transformers accelerate peft
!pip -q install "transformers==4.45.2" "accelerate==0.34.2" "peft==0.12.0" "datasets==2.21.0" "evaluate==0.4.2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 112.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 97.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.6.1 which is incompatible.


In [ ]:
from datasets import load_dataset, DatasetDict, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoConfig,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    pipeline
)
import torch
from torch.optim import Adam
import numpy as np

# Log in to Hugging Face. This is necessary for accessing Llama models.
# notebook_login()

model_id = "distilbert-base-uncased"
model_checkpoint = "distilbert-base-uncased"

# Check if a GPU is available and set the device accordingly
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

print(f"Model '{model_id}' loaded successfully on {device}.")


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Using device: cuda
Model 'distilbert-base-uncased' loaded successfully on cuda.


In [ ]:
label2id = {'Negative': 0, 'Positive': 1}
id2label = {v: k for k, v in label2id.items()}

# Now your model instantiation code will work
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
 # load dataset
dataset = load_dataset("shawhin/imdb-truncated")
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
})

In [ ]:
# display % of training data with label=1
np.array(dataset['train']['label']).sum()/len(dataset['train']['label'])

np.float64(0.5)

In [ ]:
# create tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

  # create tokenize function
def tokenize_function(examples):
      # extract text
      text = examples["text"]

      # tokenize and truncate text
      tokenizer.truncate_side = "left"
      tokenized_inputs = tokenizer(
          text,
          return_tensors="np",
          truncation=True,
          max_length=512
      )

      return tokenized_inputs

# add pad token if none exists
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

# tokenize training and validation datasets
tokenzied_dataset = dataset.map(tokenize_function, batched=True)
tokenzied_dataset

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
!pip install evaluate
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

text_list = [
    "It was good.",
    "Not a fan, don't recommend.",
    "Better than the first one.",
    "This is not worth watching even once.",
    "This one is a pass."
]

print("Untrained model predictions:")
print("-----------------------")

with torch.no_grad():
    for text in text_list:
        # Use tokenizer(...) not encode, get a dict and move each tensor to device
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        outputs = model(**inputs)
        logits = outputs.logits
        pred_id = int(torch.argmax(logits, dim=-1))
        print(f"{text} - {id2label[pred_id]}")


Untrained model predictions:
-----------------------
It was good. - Positive
Not a fan, don't recommend. - Positive
Better than the first one. - Positive
This is not worth watching even once. - Positive
This one is a pass. - Positive


In [ ]:
from peft import LoraConfig, get_peft_model
peft_config = LoraConfig(task_type="SEQ_CLS", # sequence classification
                        r=4, # intrinsic rank of trainable weight matrix
                        lora_alpha=32, # this is like a learning rate
                        lora_dropout=0.01, # probability of dropout
                        target_modules = ['q_lin']) # we apply lora to query layer

In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 628,994 || all params: 67,584,004 || trainable%: 0.9307


In [ ]:
from transformers import TrainingArguments

# hyperparameters
lr = 1e-3
batch_size = 4
num_epochs = 10
# define training arguments
training_args = TrainingArguments(
    output_dir= model_checkpoint + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    report_to=[],
    load_best_model_at_end=True,
)

In [ ]:
tokenzied_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [ ]:
# Check if you're using GPU
print(f"Device: {device}")
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")


Device: cuda
GPU available: True
GPU name: Tesla T4


In [ ]:

print("Original tokenized columns:", tokenzied_dataset["train"].column_names)

# Define the function to rename 'label' to 'labels'
def rename_and_format(example):
    example['labels'] = example['label']
    return example

# Apply the renaming
tokenzied_dataset = tokenzied_dataset.map(rename_and_format)

# Remove the columns that the model doesn't need: 'text' and the old 'label'
cleaned_dataset = tokenzied_dataset.remove_columns(['label', 'text'])

print("Cleaned dataset columns:", cleaned_dataset["train"].column_names)

Original tokenized columns: ['label', 'text', 'input_ids', 'attention_mask']


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Cleaned dataset columns: ['input_ids', 'attention_mask', 'labels']


In [ ]:
train = cleaned_dataset["train"]
eval = cleaned_dataset["validation"]

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.334013,{'accuracy': 0.882}
2,0.416400,0.513921,{'accuracy': 0.87}
3,0.416400,0.703026,{'accuracy': 0.875}
4,0.185000,0.718117,{'accuracy': 0.877}
5,0.185000,0.768981,{'accuracy': 0.884}
6,0.059300,0.777214,{'accuracy': 0.887}
7,0.059300,0.907079,{'accuracy': 0.883}
8,0.016800,0.954159,{'accuracy': 0.879}
9,0.016800,1.040653,{'accuracy': 0.881}
10,0.006100,1.020628,{'accuracy': 0.882}


TrainOutput(global_step=2500, training_loss=0.13669121685028077, metrics={'train_runtime': 475.6822, 'train_samples_per_second': 21.022, 'train_steps_per_second': 5.256, 'total_flos': 1112883852759936.0, 'train_loss': 0.13669121685028077, 'epoch': 10.0})

In [ ]:
model.to('cuda')
print("Trained model predictions:")
print("------------------------")
for text in text_list:
    inputs = tokenizer.encode(text, return_tensors="pt").to("cuda")

    logits = model(inputs).logits
    predictions = torch.max(logits, 1).indices

    print(text + " - " + id2label[predictions.tolist()[0]])

Trained model predictions:
------------------------
It was good. - Positive
Not a fan, don't recommend. - Negative
Better than the first one. - Positive
This is not worth watching even once. - Negative
This one is a pass. - Negative
